In [1]:
import requests
import pandas as pd

# World Bank indicator codes for ESG/Risk-related metrics
INDICATORS = {
    "GFDD.EI.03": "Return on Assets (ROA) (%)",
    "GFDD.SI.02": "Capital to Risk-Weighted Assets (%)",
    "GFDD.SI.01": "Capital to Assets Ratio (%)",
    "GFDD.EI.04": "Return on Equity (ROE) (%)",
    "GFDD.OI.02": "Interest Income to Total Income (%)"
}

COUNTRIES_URL = "http://api.worldbank.org/v2/country?format=json&per_page=300"
BASE_URL = "http://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json&per_page=1000"

def fetch_country_codes():
    response = requests.get(COUNTRIES_URL)
    data = response.json()
    countries = [entry['id'] for entry in data[1] if entry['region']['id'] != 'NA']
    return countries

def fetch_indicator_data(country, indicator):
    url = BASE_URL.format(country=country, indicator=indicator)
    try:
        response = requests.get(url)
        data = response.json()
        if not isinstance(data, list) or len(data) < 2 or data[1] is None:
            return []
        return [{
            "country": entry.get("country", {}).get("id"),
            "country_name": entry.get("country", {}).get("value"),
            "indicator": indicator,
            "indicator_name": INDICATORS[indicator],
            "year": entry.get("date"),
            "value": entry.get("value")
        } for entry in data[1] if entry.get("value") is not None]
    except Exception:
        return []

def main():
    countries = fetch_country_codes()
    all_data = []
    for indicator in INDICATORS:
        print(f"Fetching indicator: {INDICATORS[indicator]}")
        for country in countries:
            records = fetch_indicator_data(country, indicator)
            all_data.extend(records)
    df = pd.DataFrame(all_data)
    df.to_csv("esg_risk_data.csv", index=False)
    print(f"Collected {len(df)} rows of ESG/Risk data and saved to 'esg_risk_data.csv'")

main()


Fetching indicator: Return on Assets (ROA) (%)
Fetching indicator: Capital to Risk-Weighted Assets (%)
Fetching indicator: Capital to Assets Ratio (%)
Fetching indicator: Return on Equity (ROE) (%)
Fetching indicator: Interest Income to Total Income (%)
Collected 20649 rows of ESG/Risk data and saved to 'esg_risk_data.csv'


In [3]:
import requests
import pandas as pd

def fetch_indicator(indicator_code, name):
    url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_code}?format=json&per_page=20000"
    response = requests.get(url)
    data = response.json()[1]

    return pd.DataFrame([{
        "country": x["country"]["id"],
        "year": int(x["date"]),
        name: x["value"]
    } for x in data if x["value"] is not None])


In [4]:
df_esg = pd.read_csv("esg_risk_data.csv")
df_esg["year"] = df_esg["year"].astype(int)


In [5]:
df_gdp = fetch_indicator("NY.GDP.MKTP.CD", "gdp")
df_pop = fetch_indicator("SP.POP.TOTL", "population")

df_merged = df_esg.merge(df_gdp, on=["country", "year"], how="left")
df_merged = df_merged.merge(df_pop, on=["country", "year"], how="left")


In [6]:
df_merged.to_csv("esg_enriched.csv", index=False)
print(f"Merged dataset saved with {len(df_merged)} rows.")


Merged dataset saved with 20649 rows.


In [11]:
df_merged = df_merged.dropna(subset=["gdp", "population"])
df_merged.to_csv("esg_risk_enriched.csv", index=False)

In [9]:
%run beeper.ipynb
beeper()

Process complete!
